# 🎬 OneWord AI - Subtitle Generator (Colab)

Generate cinematic one-word subtitles from your videos using Whisper!

**Perfect for**: Reels, Shorts, TikToks

---

## 📦 Step 1: Install Dependencies

This will install Whisper and required packages.

In [ ]:
!pip install -q openai-whisper tqdm

## 📁 Step 2: Upload Your Video/Audio

Click the upload button and select your file.

In [ ]:
from google.colab import files
import os

print("📤 Upload your video/audio file:")
uploaded = files.upload()

# Get the filename
input_file = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {input_file}")

## ⚙️ Step 3: Configure Settings

Choose your preferences:

In [ ]:
# ========== CONFIGURATION ==========

# Whisper Model: 'tiny', 'base', 'small', 'medium', 'large'
MODEL = 'base'  # ⚡ Recommended for Colab

# Language: None = Auto Detect, or 'en', 'hi', 'ur', 'es'
LANGUAGE = None  # 🌍 Auto Detect

# Subtitle Mode: 'oneword', 'twoword', 'phrase'
MODE = 'oneword'  # 🎯 One word per subtitle

print("✅ Configuration set!")
print(f"   Model: {MODEL}")
print(f"   Language: {LANGUAGE or 'Auto Detect'}")
print(f"   Mode: {MODE}")

## 🧠 Step 4: Generate Subtitles

This will transcribe your video and create the SRT file!

In [ ]:
import whisper
import datetime
from pathlib import Path
from whisper.audio import load_audio

def format_timestamp(seconds):
    td = datetime.timedelta(seconds=seconds)
    total_seconds = int(td.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    secs = total_seconds % 60
    millis = int(td.microseconds / 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

# Load model
print(f"\n📦 Loading Whisper model: {MODEL}...")
model = whisper.load_model(MODEL)

# Transcribe
print(f"🧠 Transcribing {input_file}...\n")
result = model.transcribe(
    input_file,
    language=LANGUAGE,
    word_timestamps=True,
    verbose=True
)

# Generate SRT
output_file = Path(input_file).stem + f"_{MODE}.srt"
print(f"\n✍ Writing subtitles to: {output_file}")

with open(output_file, "w", encoding="utf-8") as f:
    counter = 1
    
    if MODE == 'oneword':
        for segment in result["segments"]:
            for word_data in segment["words"]:
                word = word_data["word"].strip().replace(",", "")
                if not word:
                    continue
                start = word_data["start"]
                end = word_data["end"]
                f.write(f"{counter}\n")
                f.write(f"{format_timestamp(start)} --> {format_timestamp(end)}\n")
                f.write(f"{word}\n\n")
                counter += 1
    
    elif MODE == 'twoword':
        for segment in result["segments"]:
            words = segment["words"]
            i = 0
            while i < len(words):
                word_group = words[i:i+2]
                text = " ".join([w["word"].strip().replace(",", "") for w in word_group if w["word"].strip()])
                if not text:
                    i += 2
                    continue
                start = word_group[0]["start"]
                end = word_group[-1]["end"]
                f.write(f"{counter}\n")
                f.write(f"{format_timestamp(start)} --> {format_timestamp(end)}\n")
                f.write(f"{text}\n\n")
                counter += 1
                i += 2
    
    elif MODE == 'phrase':
        for segment in result["segments"]:
            text = segment["text"].strip()
            if not text:
                continue
            start = segment["start"]
            end = segment["end"]
            f.write(f"{counter}\n")
            f.write(f"{format_timestamp(start)} --> {format_timestamp(end)}\n")
            f.write(f"{text}\n\n")
            counter += 1

print(f"\n✅ Success! SRT file created: {output_file}")

## 📥 Step 5: Download Your SRT

Download the subtitle file to your device!

In [ ]:
from google.colab import files

print("📥 Downloading SRT file...")
files.download(output_file)
print("\n✅ Done! Check your Downloads folder.")

---

## 🎉 All Done!

Import your SRT file into:
- **CapCut**: Text → Local Captions → Upload
- **VN Editor**: Text → SRT → Import
- **Premiere Pro**: File → Import → Captions

### 💡 Tips:
- Use **Tiny** model for speed, **Small** for accuracy
- **One Word** mode = Each word appears separately (best for high-energy content)
- **Two Word** mode = Punchy two-word chunks
- **Phrase** mode = Full sentences

---

**Made with ❤️ by [@ambrish.yadav.1](https://instagram.com/ambrish.yadav.1)**

⭐ Star the repo: [github.com/ambrish-yadav/minimalist-one-word-subtitle-generator](https://github.com/ambrish-yadav/minimalist-one-word-subtitle-generator)